In [1]:
import pandas as pd
import numpy as np

In [2]:
import torch
import sklearn

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
# 1. test에는 결측값이 없어도 train은 결측값이 있을 수 있다. 모두 있다고 가정하고 처리 
# 2. Binary classification에서는 layer를 줄일 필요가 있다(2개 정도가 적당)

In [5]:
# 랜덤 시드 설정
torch.manual_seed(1)
if device == "cuda" :
    torch.cuda.manual_seed_all(1)

In [6]:
# csv 데이터 로드
train = pd.read_csv("2021-ai-midterm-p5/train.csv")
test = pd.read_csv("2021-ai-midterm-p5/test.csv")
submission = pd.read_csv("2021-ai-midterm-p5/submit_sample.csv")

# 데이터 확인
print(train)
print(train.info())
print(train.describe())

      index  Unnamed: 0  customerID  gender  SeniorCitizen Partner Dependents  \
0         0        1869  7010-BRBUU    Male              0     Yes        Yes   
1         1        4528  9688-YGXVR  Female              0      No         No   
2         2        6344  9286-DOJGF  Female              1     Yes         No   
3         3        6739  6994-KERXL    Male              0      No         No   
4         4         432  2181-UAESM    Male              0      No         No   
...     ...         ...         ...     ...            ...     ...        ...   
4783   5981        3772  0684-AOSIH    Male              0     Yes         No   
4784   5982        5191  5982-PSMKW  Female              0     Yes        Yes   
4785   5983        5226  8044-BGWPI    Male              0     Yes        Yes   
4786   5984        5390  7450-NWRTR    Male              1      No         No   
4787   5985         860  4795-UXVCJ    Male              0      No         No   

      tenure PhoneService M

In [7]:
# 데이터 전처리
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
        
# object -> int
for col in test.columns:
    if test[col].dtype == "object" or train[col].dtype == "object":
        test[col] = test[col].astype(str)
        train[col] = train[col].astype(str)
        label = pd.concat([test[col],train[col]],axis=0)
        le.fit(label)
        test[col]=le.transform(test[col])
        train[col]= le.transform(train[col])
train['Churn'] = le.fit_transform(train['Churn'])
print(train.info())
print(test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4788 entries, 0 to 4787
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   index             4788 non-null   int64  
 1   Unnamed: 0        4788 non-null   int64  
 2   customerID        4788 non-null   int32  
 3   gender            4788 non-null   int32  
 4   SeniorCitizen     4788 non-null   int64  
 5   Partner           4788 non-null   int32  
 6   Dependents        4788 non-null   int32  
 7   tenure            4788 non-null   int64  
 8   PhoneService      4788 non-null   int32  
 9   MultipleLines     4788 non-null   int32  
 10  InternetService   4788 non-null   int32  
 11  OnlineSecurity    4788 non-null   int32  
 12  OnlineBackup      4788 non-null   int32  
 13  DeviceProtection  4788 non-null   int32  
 14  TechSupport       4788 non-null   int32  
 15  StreamingTV       4788 non-null   int32  
 16  StreamingMovies   4788 non-null   int32  


In [8]:
# 사용하지 않은 컬럼을 제거하여 Train, Test 데이터 설정
x_train = np.array(train.drop(["index","Unnamed: 0","Churn","customerID"],axis=1))
x_test = np.array(test.drop(["index","Unnamed: 0","customerID"],axis=1))
y_train = np.array(train["Churn"])

In [9]:
# 데이터 전처리
# 값의 범위가 크므로 정규화가 필요

from sklearn.preprocessing import StandardScaler, MinMaxScaler
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [10]:
# 데이터 모양 확인
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(len(np.unique(y_train)))

(4788, 19)
(4788,)
(1198, 19)
2


In [11]:
# 데이터 Tensor에 올리기
x_train = torch.Tensor(x_train).to(device)
x_test = torch.Tensor(x_test).to(device)
y_train = torch.Tensor(y_train).to(device)

In [12]:
# 레이어 생성
layer1 = torch.nn.Linear(x_train.shape[1],64,bias=True).to(device)
layer2 = torch.nn.Linear(64,1,bias=True).to(device)
relu = torch.nn.ReLU().to(device)
dropout = torch.nn.Dropout(0.2).to(device)
sigmoid = torch.nn.Sigmoid()

In [13]:
# 레이어 가중치 초기화
torch.nn.init.xavier_normal_(layer1.weight)
torch.nn.init.xavier_normal_(layer2.weight)


Parameter containing:
tensor([[-1.1121e-01, -1.1281e-01, -1.1262e-01, -2.5384e-01,  7.8001e-02,
         -1.0652e-01, -1.5559e-01, -1.6634e-01, -6.3910e-04, -8.0629e-02,
          6.7537e-02,  2.1359e-01,  2.6220e-01,  9.1700e-03, -2.3355e-01,
          1.9205e-01, -1.1461e-01,  2.8988e-01, -1.0759e-01, -8.9361e-02,
          2.1727e-01, -5.4712e-02,  8.9871e-03, -1.2963e-02,  1.9585e-01,
         -9.9288e-02,  3.0081e-02,  1.1173e-01,  5.1985e-02,  6.1508e-02,
          6.8215e-02, -6.4534e-02, -6.0171e-02,  5.2827e-02,  1.2673e-01,
         -1.6610e-01,  3.6227e-02,  1.1820e-01,  1.2187e-01, -2.7108e-01,
          2.3089e-01,  4.1204e-01,  4.0068e-04, -2.5482e-01,  2.0463e-01,
         -9.3607e-02,  1.7352e-01, -1.9064e-01, -2.6712e-01,  1.2376e-01,
         -3.3725e-01, -7.2971e-02,  1.6691e-01,  8.8490e-03,  2.1699e-01,
          1.6944e-02, -3.4097e-01, -2.2497e-01,  1.6907e-01,  1.6945e-01,
          1.1895e-01, -1.2339e-01,  2.9733e-01, -3.8142e-02]], device='cuda:0',
       req

In [14]:
# 모델 생성
model = torch.nn.Sequential(layer1, relu,dropout,
                           layer2,sigmoid).to(device)

In [15]:
# 학습 파라미터 설정
lr = 1e-2
epochs = 100

In [16]:
# 손실 함수, 최적화 함수 정의
loss = torch.nn.BCELoss().to(device)
optim = torch.optim.Adam(model.parameters(), lr=lr)

In [17]:
# 학습
model.train()
for epoch in range(epochs+1):
    optim.zero_grad()
    h = model(x_train)
    cost = loss(h,y_train.unsqueeze(1))
    cost.backward()
    optim.step()
    if epoch % (epochs/10) == 0:
        predict = h > torch.Tensor([0.5]).to(device)
        predict = predict.int()
        predict = predict == y_train.unsqueeze(1)
        acc = sum(predict)/len(y_train.unsqueeze(1))
        print(epoch, cost.item(),acc.item())

0 0.6764297485351562 0.5680869221687317
10 0.4831795394420624 0.7690058350563049
20 0.4485970437526703 0.7794486284255981
30 0.435477077960968 0.7932330965995789
40 0.425169974565506 0.7982456088066101
50 0.4236586093902588 0.7999164462089539
60 0.41738712787628174 0.7982456088066101
70 0.4135073125362396 0.8009607195854187
80 0.4133182168006897 0.8080618381500244
90 0.40992072224617004 0.8082706928253174
100 0.4077272415161133 0.8116123676300049


In [18]:
with torch.no_grad():
    model.eval()
    predict = model(x_test)
    predict = predict >= torch.Tensor([0.5]).to(device)
    predict = predict.float()
submission["Churn"] = predict.cpu().detach().numpy()
print(submission)

      index  Churn
0         5    0.0
1        10    0.0
2        13    0.0
3        18    1.0
4        20    0.0
...     ...    ...
1193   5962    0.0
1194   5968    0.0
1195   5975    0.0
1196   5977    0.0
1197   5978    1.0

[1198 rows x 2 columns]


In [19]:
submission.to_csv("submission.csv",index=False)